In [ ]:
from google.cloud import storage
from dotenv import load_dotenv, set_key
import os 

load_dotenv('/workspaces/spotify_insights/src/transform/.env') 
set_key(".env", "SPOTIFY_RAW_PARQUET_RELATIVE_PATH", "raw-tables")


SPOTIFY_SOURCE_PATH=/workspaces/spotify_insights/0_tmp_data/new
SPOTIFY_SOURCE_BUCKET=spotify-insights-source-data
SPOTIFY_RAW_JSON_RELATIVE_PATH=raw
SPOTIFY_PIPELINE_BUCKET=spotify-insights-pipeline-data
SPOTIFY_RAW_PARQUET_RELATIVE_PATH=raw-tables
GOOGLE_APPLICATION_CREDENTIALS=/workspaces/spotify_insights/.gcp.auth/spotify-insights-444509-9f9af456ef0b.json

In [2]:
source_bucket_name= os.getenv("SPOTIFY_SOURCE_BUCKET")
source_path= os.getenv("SPOTIFY_RAW_JSON_RELATIVE_PATH")    
destination_bucket_name= os.getenv("SPOTIFY_PIPELINE_BUCKET")
destination_path= os.getenv("SPOTIFY_RAW_PARQUET_RELATIVE_PATH")  
#destination_path = destination_path[1:]
destination_path

'raw-tables'

In [3]:

def get_file_components(blob_name):    
    def get_safe(list, index, default=''): 
        return list[index] if len(list) > abs(index) else default    
    components =blob_name.split('/')
    username = get_safe(components, -3) 
    date = get_safe(components, -2) 
    filename = get_safe(components, -1) 
    #print(len(components))
    message = "Processing requires path form: /folders..../<username>/<date>/<filename>"
    if filename == "":
        print(f"{message}. Filename is empty")
    elif username == "" or date == "":
        print(f"{message}. username '{username}' or date '{date}' is empty")
    
    
    return (username, date, filename)



In [4]:
get_file_components('raw/joseph-higaki/2024-12-10/Streaming_History_Audio_2014-2016_0.json')

('joseph-higaki', '2024-12-10', 'Streaming_History_Audio_2014-2016_0.json')

In [5]:


def iterate_files_from_gcs():
    storage_client = storage.Client()
    bucket = storage_client.bucket(source_bucket_name)
    for blob_item in bucket.list_blobs():    
        #print(blob_item)
        username, date, filename = get_file_components(blob_item.name)
            
        if 'Streaming_History_Audio_2024_5.json' in  blob_item.name:
            yield (username, date, filename, blob_item.download_as_string())
        #blob = bucket.blob(blob_item.name)
        #json = blob.download_as_bytes()
#        print(json)


In [6]:
import os

def get_destination_full_path(destination_bucket_name, destination_path, username="", date="", filename="") -> str:
    path = f"gs://{destination_bucket_name}/{destination_path}"
    if username: path += f"/{username}"
    if date: path += f"/{date}"
    if filename:
        basename = os.path.basename(filename)
        name, _ = os.path.splitext(basename)
        path += f"/{name}.parquet"
    return path

# Example usage
username = "joseph-higaki"
date = "2024-12-10"
filename = "Streaming_History_Audio_2022-2024_4.json"

full_path = get_destination_full_path(destination_bucket_name, destination_path, username, date, filename)
print(full_path)  # Output: gs://my-bucket/my-folder/joseph-higaki/2024-12-10/Streaming_History_Audio_2022-2024_4.parquet


full_path = get_destination_full_path(destination_bucket_name, destination_path)
print(full_path)  # Output: gs://my-bucket/my-folder/joseph-higaki/2024-12-10/Streaming_History_Audio_2022-2024_4.parquet

gs://spotify-insights-pipeline-data/raw-tables/joseph-higaki/2024-12-10/Streaming_History_Audio_2022-2024_4.parquet
gs://spotify-insights-pipeline-data/raw-tables


In [7]:
import logging
logging.basicConfig(level=logging.DEBUG)

# Upload to GCS manually setting the folders

In [9]:
import pandas as pd
from io import BytesIO
from IPython.display import display, HTML

In [187]:




for username, date, filename, json_content in iterate_files_from_gcs():
    #print("see", username, date, filename)
    #print(json_content[:8])
    df = pd.read_json(BytesIO(json_content), dtype=str)
    df['user_name'] = username
    df['snapshot_date'] = date
    #display(df)
    #print(df.dtypes)
    destination_full_path= get_destination_full_path(destination_bucket_name, destination_path, username, date, filename)
    print(destination_full_path)
    df.to_parquet(destination_full_path, engine="pyarrow")
    #df.to_parquet("gs://spotify-insights-pipeline-data/Streaming_History_Audio_2014-2016_0.parquet", engine="pyarrow")
    
    


DEBUG:google.auth._default:Checking /workspaces/spotify_insights/.gcp.auth/spotify-insights-444509-9f9af456ef0b.json for explicit credentials as part of auth process...
DEBUG:google.auth._default:Checking /workspaces/spotify_insights/.gcp.auth/spotify-insights-444509-9f9af456ef0b.json for explicit credentials as part of auth process...
DEBUG:urllib3.util.retry:Converted retries value: 3 -> Retry(total=3, connect=None, read=None, redirect=None, status=None)
DEBUG:google.auth.transport.requests:Making request: POST https://oauth2.googleapis.com/token
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): oauth2.googleapis.com:443
DEBUG:urllib3.connectionpool:https://oauth2.googleapis.com:443 "POST /token HTTP/11" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): storage.googleapis.com:443
DEBUG:urllib3.connectionpool:https://storage.googleapis.com:443 "GET /storage/v1/b/spotify-insights-source-data/o?projection=noAcl&prettyPrint=false HTTP/11" 200 7089
DEB

gs://spotify-insights-pipeline-data/raw-tables/joseph-higaki/2024-12-10/Streaming_History_Audio_2024_5.parquet


In [188]:
read_from_gcs = pd.read_parquet("gs://spotify-insights-pipeline-data/raw-tables/joseph-higaki/2024-12-10/Streaming_History_Audio_2024_5.parquet", engine="pyarrow")
display(read_from_gcs)

,ts,platform,ms_played,conn_country,ip_addr,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,episode_name,...,spotify_episode_uri,reason_start,reason_end,shuffle,skipped,offline,offline_timestamp,incognito_mode,user_name,snapshot_date
0,2024-06-07T17:58:17Z,android,173186,PE,200.25.47.107,Ouvrez les frontières - Acoustic Version,Tiken Jah Fakoly,Ouvrez les frontières,spotify:track:2Qv13cE9Cx01hRLjZ9yQvv,None,...,None,trackdone,trackdone,False,False,False,1717782926,False,joseph-higaki,2024-12-10
1,2024-06-07T18:03:07Z,android,289026,PE,200.25.47.107,Layla - Acoustic Live,Eric Clapton,Unplugged (Deluxe Edition) (Live),spotify:track:5phbwzg5d2MqNAoc0435Zu,None,...,None,trackdone,trackdone,False,False,False,1717783099,False,joseph-higaki,2024-12-10
2,2024-06-07T18:07:26Z,android,260080,PE,200.25.47.107,Nuestra Canción,Monsieur Periné,Caja de Música,spotify:track:5reQI13tWWYDLMrGcUF4Mk,None,...,None,trackdone,trackdone,False,False,False,1717783389,False,joseph-higaki,2024-12-10
3,2024-06-07T18:12:34Z,android,306560,PE,200.25.47.107,I'll Be Missing You (feat. Faith Evans & 112) ...,Diddy,No Way Out,spotify:track:1yy2DlSDtEt90d54rPDPXz,None,...,None,trackdone,trackdone,False,False,False,1717783649,False,joseph-higaki,2024-12-10
4,2024-06-07T18:17:41Z,android,306066,PE,200.25.47.107,Where Did You Sleep Last Night - Live,Nirvana,MTV Unplugged In New York,spotify:track:551qy5vUgrUfEUc4dCNfht,None,...,None,trackdone,trackdone,False,False,False,1717783957,False,joseph-higaki,2024-12-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4162,2024-12-04T06:55:39Z,android,282653,ES,79.153.169.248,Voodoo,Godsmack,Godsmack,spotify:track:5uGsG0LfotfWDq6hql4h53,None,...,None,trackdone,trackdone,True,False,True,1733295056,False,joseph-higaki,2024-12-10
4163,2024-12-04T06:58:59Z,android,199906,ES,79.153.169.248,Last Resort,Papa Roach,Infest,spotify:track:5W8YXBz9MTIDyrpYaCg2Ky,None,...,None,trackdone,trackdone,True,False,True,1733295339,False,joseph-higaki,2024-12-10
4164,2024-12-04T07:04:05Z,android,304840,ES,79.153.169.248,Uprising,Muse,The Resistance,spotify:track:4VqPOruhp5EdPBeR92t6lQ,None,...,None,trackdone,trackdone,True,False,False,1733295539,False,joseph-higaki,2024-12-10
4165,2024-12-04T07:26:58Z,android,148181,ES,79.117.89.98,Killing In The Name,Rage Against The Machine,Rage Against The Machine - XX (20th Anniversar...,spotify:track:59WN2psjkt1tyaxjspN8fp,None,...,None,trackdone,unknown,True,False,False,1733295845,False,joseph-higaki,2024-12-10


# Upload to GCS partitioning by username and snapshot

In [10]:
import pyarrow as pa
import pyarrow.parquet as pq

for username, date, filename, json_content in iterate_files_from_gcs():
    #print("see", username, date, filename)
    #print(json_content[:8])
    df = pd.read_json(BytesIO(json_content), dtype=str)
    df['user_name'] = username
    df['snapshot_date'] = date

    table = pa.Table.from_pandas(df)
    destination_full_path= get_destination_full_path(destination_bucket_name, destination_path)
    print(destination_full_path)
    pq.write_to_dataset(
        table,
        root_path=destination_full_path,
        partition_cols=['user_name', 'snapshot_date']
    )
    #display(df)
    #print(df.dtypes)
    
    #print(destination_full_path)
    #df.to_parquet(destination_full_path, engine="pyarrow")

DEBUG:google.auth._default:Checking /workspaces/spotify_insights/.gcp.auth/spotify-insights-444509-9f9af456ef0b.json for explicit credentials as part of auth process...
DEBUG:google.auth._default:Checking /workspaces/spotify_insights/.gcp.auth/spotify-insights-444509-9f9af456ef0b.json for explicit credentials as part of auth process...
DEBUG:urllib3.util.retry:Converted retries value: 3 -> Retry(total=3, connect=None, read=None, redirect=None, status=None)
DEBUG:google.auth.transport.requests:Making request: POST https://oauth2.googleapis.com/token
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): oauth2.googleapis.com:443
DEBUG:urllib3.connectionpool:https://oauth2.googleapis.com:443 "POST /token HTTP/11" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): storage.googleapis.com:443
DEBUG:urllib3.connectionpool:https://storage.googleapis.com:443 "GET /storage/v1/b/spotify-insights-source-data/o?projection=noAcl&prettyPrint=false HTTP/11" 200 7089
DEB

gs://spotify-insights-pipeline-data/raw-tables


In [12]:
print(df.dtypes)

ts                                   object
platform                             object
ms_played                            object
conn_country                         object
ip_addr                              object
master_metadata_track_name           object
master_metadata_album_artist_name    object
master_metadata_album_album_name     object
spotify_track_uri                    object
episode_name                         object
episode_show_name                    object
spotify_episode_uri                  object
reason_start                         object
reason_end                           object
shuffle                              object
skipped                              object
offline                              object
offline_timestamp                    object
incognito_mode                       object
user_name                            object
snapshot_date                        object
dtype: object


In [13]:
read_from_gcs_folder = pd.read_parquet("gs://spotify-insights-pipeline-data/raw-tables", engine="pyarrow")
display(read_from_gcs_folder)

ArrowTypeError: Unable to merge: Field user_name has incompatible types: string vs dictionary<values=string, indices=int32, ordered=0>